In [4]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

#### План отчета:
&nbsp;

**1.	Постановка задачи и пути решения.**

&#8195;Зачем нужно? Почему важно? Как решать?


**2.	Первичный анализ данных**

&#8195;Что имеем? Стратегия работы

**3.	Предобработка выборки**

&#8195;Как преобразуем выборку что бы следовать стратегии работы?
>Можно выполнить сравнение (с применением виджетов) работы функции, считающей признаки для выборки. С добавлением значений по одному в соответствующую ячейку dataframe и добавлением по-столбцам (разница ~20 раз)

**4.	Проверка гипотез**

&#8195;Какие показатели значимо выделают одних пользователей от других? Почему?

**5.	Features development**

&#8195;Какие фичи будем пробовать? Почему? Что получилось?

**6.	Обучение и каггл**

&#8195;Почему бинарная классификация показывает себя хорошо, а многоклассовая дает плохие показатели метрик. Сравим метрики на каггле при разном количестве признаков

**7.	Многоклассовая классификация с Ворпал вэббит и SGDlogreg**

&#8195;Почему это в принципе так неэффективно и что менее неэффективно.

**8.	Вывод по решению задачи на данный момент, возможности ее выкатывания в продакшн, а так же дальнейшая стратегия по исследованию данной темы.**


### Содержание
&nbsp;
1. [Цель и задачи проекта. Описание исходных и обработанных данных и признаков](#1part)
2. [Описание данных и их подготовка](#2part)
 - [Описание данных](#2.1part)
  - [Создание сессий](#2.2part)
  - [Применение мешка слов](#2.3part)
  - [Варьирование длины сессии и скользящее окно](#2.4part)
3. [Первичный анализ данных. Проверка гипотез](#3part)
4. [Разработка признаков и их визуальный анализ](#4part);
  - [Временные признаки и количество уникальных сайтов](#4.1part)
  - [Вывод для каждого пользователя по построенным признакам](#4.2part)
  - [Признаки "работает или развлекается пользователь"](#4.3part)
5. [Сравнение алгоритмов и подбор модели](#5part);
  - [Cравнение нескольких алгоритмов на сессиях из 10 сайтов](#5.1part)
  - [Выбор параметров – длины сессии и ширины окна](#5.2part)
  - [Идентификация конкретного пользователя и кривая обучения](#5.3part)
6. [Соревнование Kaggle](#6part)
  - [Анализ и подготовка данных](#6.1part)
  - [Первый сабмит](#6.2part)
  - [Второй сабмит](#6.3part)
  - [Третий сабмит](#6.4part)
7. Работа с Worpal Wabbit
8. Итоги, выводы и описание дальнейшей работы

<a id='1part'></a>
### 1. Цель и задачи проекта. Описание исходных и обработанных данных и признаков
&nbsp;

1.1 Цель проекта

&#8195;Проект направлен на решение задачи идентификации пользователя по его поведению в сети Интернет. Задача как таковая является широкой в смысле выбора используемых признаков и стратегии. К примеру идентифицировать пользователя можно как по тому какие сайты в какое время он посещает, так и по тому как он это делает - какими разделами сайта он пользуется, как быстро отвечает на сообщения в социальных сетях, насколько длинны оставляемые им комментарии, или он предпочитает их не оставлять и т.д.

&#8195;Целью данного проекта является построение алгоритма идентификации пользователя по имеющимся данным с последовательностями посещенных пользователями сайтов и времени посещения каждого из них.

&nbsp;

1.2 Задачи проекта

- Предобработка данных, подготовка их к классификации
- Первичный и визуальный анализ
- Разработка признаков и их описание
- Сравнение семейств моделей классификации и подбор параметров выбранной модели
- Оценка метрик и вывод по проделанной работе
- Прогноз по дальнейшей работе


<a id='2part'></a>
### 2. Описание данных и их подготовка
&nbsp;



<a id='2.1part'></a>
#### Описание данных

&nbsp;

&#8195;Имеются данные с прокси-серверов Университета Блеза Паскаля.

&#8195;Для каждого пользователя заведен csv-файл с названием user****.csv (где вместо звездочек – 4 цифры, соответствующие ID пользователя), а в нем посещения сайтов записаны в следующем формате: 

&nbsp;

<center>*timestamp, посещенный веб-сайт*</center>
&nbsp;

&#8195;В дальнейшем будет использоваться параллельно 2 выборки: по 10 и по 150  пользователям. Для 10 пользователей будет отлаживаться код, для 150 будет использоваться рабочая версия.

&#8195;Вид исходных данных приведен ниже. 

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-yw4l">site</th>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:07</td>
    <td class="tg-yw4l">fpdownload2.macromedia.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:17</td>
    <td class="tg-yw4l">www.laposte.net</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:17</td>
    <td class="tg-yw4l">www.google.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:18</td>
    <td class="tg-yw4l">www.laposte.net</td>
  </tr>
</table>

<a id='2.2part'></a>
#### Создание сессий

&nbsp;

&#8195;На данном этапе поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. 

&#8195;Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов. Целевым классом будет id пользователя.

&#8195;Отображение сайтов в их индексы выглядит так:
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>


&#8195;Для подготовки обучающей выборки реализована функция  prepare_train_set, которая принимает на вход путь к каталогу с csv-файлами path_to_csv_files и параметр session_length – длину сессии, а возвращает 2 объекта: 
- DataFrame, в котором строки соответствуют уникальным сессиям из session_length сайтов, session_length столбцов – индексам этих session_length сайтов и последний столбец – ID пользователя
- частотный словарь сайтов вида {'site_string': [site_id, site_freq]}

После применения функции получаем таблицу вида:

&nbsp;



<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site1</th>
      <th>site2</th>
      <th>site3</th>
      <th>site4</th>
      <th>site5</th>
      <th>site6</th>
      <th>site7</th>
      <th>site8</th>
      <th>site9</th>
      <th>site10</th>
      <th>user_id</th>
    </tr>
    <tr>
      <th>session_id</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>3</td>
      <td>2</td>
      <td>2</td>
      <td>9</td>
      <td>2</td>
      <td>1</td>
      <td>7</td>
      <td>5</td>
      <td>8</td>
      <td>10</td>
      <td>0001</td>
    </tr>
    <tr>
      <th>1</th>
      <td>3</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0001</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>2</td>
      <td>6</td>
      <td>6</td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0002</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>1</td>
      <td>2</td>
      <td>1</td>
      <td>2</td>
      <td>1</td>
      <td>1</td>
      <td>5</td>
      <td>11</td>
      <td>4</td>
      <td>0003</td>
    </tr>
    <tr>
      <th>4</th>
      <td>4</td>
      <td>1</td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0003</td>
    </tr>
  </tbody>
</table>

И словарь вида:

     {'google.com': [1, 9],
     'oracle.com': [2, 8],
     'vk.com': [3, 3],
     'meduza.io': [4, 3],
     'mail.google.com': [5, 2],
     'football.kulichki.ru': [6, 2],
     'accounts.google.com': [7, 1],
     'apis.google.com': [8, 1],
     'geo.mozilla.org': [9, 1],
     'plus.google.com': [10, 1],
     'yandex.ru': [11, 1]}

<a id='2.3part'></a>
#### Применение мешка слов

&nbsp;

&#8195;Далее, что бы наши категориальные признаки с индексами сайтов приобрели смысл в рамках машинного обучения применим идею мешка слов. Точнее говоря не только идею, но и фактически преобразуем матрицы признаков в матрицу, где число столбцов равно количеству уникальных индексов сайтов, а в ячейках указано количество посещений того или иного сайта в рамках сессии.

&#8195;Делать это будем с помощью разреженных матриц Scipy – csr_matrix. 

&#8195;Напишем функцию sparsecreator, которая приобразовывает массив с категориальными признаками в разряженную матрицу с вещественными.

&#8195;Для 150 пользователей получим матрицу размером 137019x27797, где 27797 - число уникальных сайтов в выборке.

&#8195;Полученная матрица:

    matrix([[4, 2, 0, ..., 0, 0, 0],
            [0, 1, 0, ..., 0, 0, 0],
            [5, 1, 0, ..., 0, 0, 0],
            ...,
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

<a id='2.4part'></a>
#### Варьирование длины сессии и скользящее окно

&nbsp;

&#8195;Пока мы брали последовательности из 10 сайтов. Далее сделаем число сайтов в сессии параметром, так что бы в дальнейшем сравнить модели классификации, обученные на разных выборках – с 5, 7, 10 и 15 сайтами в сессии. А так же применим идею скользящего окна – сессии будут перекрываться.

&#8195;Разработаем функцию prepare_sparse_train_set_window.

&#8195;Аргументы:
- path_to_csv_files – путь к каталогу с csv-файлами
- site_freq_path – путь к pickle-файлу с частотным словарем, полученным в 1 части проекта
- session_length – длина сессии (параметр)
- window_size – ширина окна (параметр)

&#8195;Значения:
- разреженная матрица X_sparse (двухмерная Scipy.sparse.csr_matrix), в которой строки соответствуют сессиям из session_length сайтов, а max(site_id) столбцов – количеству посещений site_id в сессии.
- вектор y (Numpy array) "ответов" в виде ID пользователей, которым принадлежат сессии из X_sparse

&#8195;Запустим созданную функцию со значениям параметра session_length (15, 10, 7, 5) и значениям параметра window_size (10, 7, 5) для выборки 10 и 150 пользователей. Сериализуем полученные 16 разреженных матриц (обучающие выборки) и вектора с метками целевого класса в файлы.

<a id='3part'></a>
### 3. Первичный анализ данных. Проверка гипотез


&#8195;Проверка гипотез будет осуществляться на матрице сессии-сайты с длинной сессии и окна 10 для выборки 10 пользователей.

&#8195;Так выглядит ее структура:

&nbsp;
    
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site1</th>
      <th>site2</th>
      <th>site3</th>
      <th>site4</th>
      <th>site5</th>
      <th>site6</th>
      <th>site7</th>
      <th>site8</th>
      <th>site9</th>
      <th>site10</th>
      <th>user_id</th>
    </tr>
    <tr>
      <th>session_id</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>192</td>
      <td>576</td>
      <td>133</td>
      <td>3</td>
      <td>133</td>
      <td>133</td>
      <td>3</td>
      <td>133</td>
      <td>203</td>
      <td>133</td>
      <td>31</td>
    </tr>
    <tr>
      <th>1</th>
      <td>421</td>
      <td>193</td>
      <td>677</td>
      <td>254</td>
      <td>133</td>
      <td>31</td>
      <td>393</td>
      <td>3403</td>
      <td>217</td>
      <td>55</td>
      <td>31</td>
    </tr>
    <tr>
      <th>2</th>
      <td>55</td>
      <td>3</td>
      <td>55</td>
      <td>55</td>
      <td>5</td>
      <td>293</td>
      <td>421</td>
      <td>333</td>
      <td>899</td>
      <td>55</td>
      <td>31</td>
    </tr>
    <tr>
      <th>3</th>
      <td>473</td>
      <td>3570</td>
      <td>473</td>
      <td>55</td>
      <td>55</td>
      <td>55</td>
      <td>55</td>
      <td>946</td>
      <td>200</td>
      <td>123</td>
      <td>31</td>
    </tr>
    <tr>
      <th>4</th>
      <td>343</td>
      <td>55</td>
      <td>5</td>
      <td>3399</td>
      <td>258</td>
      <td>212</td>
      <td>3498</td>
      <td>2095</td>
      <td>674</td>
      <td>2095</td>
      <td>31</td>
    </tr>
  </tbody>
</table>

&#8195;Распределение целевого класса:
    
    128    2796
    39     2204
    207    1868
    127    1712
    237    1643
    33     1022
    50      802
    31      760
    100     720
    241     534
    Name: user_id, dtype: int64
    
&#8195;Посчитаем распределение числа уникальных сайтов в каждой сессии из 10 посещенных подряд сайтов:

    7     2308
    6     2197
    8     2046
    5     1735
    9     1394
    2     1246
    4     1163
    3      894
    10     651
    1      427
    dtype: int64
 
 &nbsp;
 
![](image1.png)

&nbsp;

&#8195;Для проверки нормальности рапределения числа уникальных сайтов воспользуемся критерием Шапиро-Уилка и построим QQ-график.

&nbsp;

 ![](image2.png)
 
 &nbsp;
 
&#8195;Критерий на данной выборке показывает p-value близкое к нулю, на QQ-графике видны длинные хвосты, т.е. имеется перекос в сторону больших и минимальных значений. Число уникальных сайтов распределено не нормально.

&#8195;Проверим с помощью биномиального критерия для доли, что доля случаев, когда пользователь повторно посетил какой-то сайт (то есть число уникальных сайтов в сессии < 10) велика: больше 95%.

&#8195;Полученное p-value 0.022

&#8195;95% доверительный интервал Уилсона для доли случаев, когда пользователь повторно посетил какой-то сайт:

(0.9501028841411286, 0.9570527377232229)

&#8195;На основании этих результатов можно уверенно предположить, что пользователи склонны посещать сайты (сайт) повторно в рамках сессии.

&#8195;Постром распределение частоты посещения сайтов для сайтов, которые были посещены как минимум 1000 раз.

&nbsp;

![ss](image3.png)

&#8195;Построим 95% доверительный интервал для средней частоты появления сайта в выборке на основе bootstrap. Используем столько же bootstrap-подвыборок, сколько сайтов оказалось в исходной выборке по 10 пользователям (получим массив 4913 Х 4913).

&#8195;Полученный интервал средней частоты появления сайта в выборке: (22.51524527, 35.76303684)

<a id='4part'></a>
### 4. Разработка признаков и их визуальный анализ

<a id='4.1part'></a>
#### Временные признаки и количество уникальных сайтов

&nbsp;

&#8195;У нас уже есть признаки - индексы сайтов в сессиях. Добавим к ним следующие признаки:
- session_timespan – продолжительность сессии (разница между максимальным и минимальным временем посещения сайтов в сессии, в секундах)
- #unique_sites – число уникальных сайтов в сессии
- start_hour – час начала сессии (то есть час в записи минимального timestamp среди десяти)
- day_of_week – день недели (то есть день недели в записи минимального timestamp среди десяти)
- time_diff# где # - номер сайта, временной промежуток между посещениями сайтов (в секундах)

&#8195;Получим таблицу следующего вида:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site1</th>
      <th>site2</th>
      <th>site3</th>
      <th>site4</th>
      <th>site5</th>
      <th>site6</th>
      <th>site7</th>
      <th>site8</th>
      <th>site9</th>
      <th>site10</th>
      <th>time_diff1</th>
      <th>time_diff2</th>
      <th>time_diff3</th>
      <th>time_diff4</th>
      <th>time_diff5</th>
      <th>time_diff6</th>
      <th>time_diff7</th>
      <th>time_diff8</th>
      <th>time_diff9</th>
      <th>session_timespan</th>
      <th>#unique_sites</th>
      <th>start_hour</th>
      <th>day_of_week</th>
      <th>target</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>192</td>
      <td>576</td>
      <td>133</td>
      <td>3</td>
      <td>133</td>
      <td>133</td>
      <td>3</td>
      <td>133</td>
      <td>203</td>
      <td>133</td>
      <td>10</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>20</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>33</td>
      <td>5</td>
      <td>8</td>
      <td>4</td>
      <td>31</td>
    </tr>
    <tr>
      <th>1</th>
      <td>421</td>
      <td>193</td>
      <td>677</td>
      <td>254</td>
      <td>133</td>
      <td>31</td>
      <td>393</td>
      <td>3403</td>
      <td>217</td>
      <td>55</td>
      <td>1</td>
      <td>0</td>
      <td>163</td>
      <td>105</td>
      <td>0</td>
      <td>1</td>
      <td>3</td>
      <td>3</td>
      <td>8</td>
      <td>284</td>
      <td>10</td>
      <td>8</td>
      <td>4</td>
      <td>31</td>
    </tr>
    <tr>
      <th>2</th>
      <td>55</td>
      <td>3</td>
      <td>55</td>
      <td>55</td>
      <td>5</td>
      <td>293</td>
      <td>421</td>
      <td>333</td>
      <td>899</td>
      <td>55</td>
      <td>0</td>
      <td>14</td>
      <td>1</td>
      <td>242</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>258</td>
      <td>7</td>
      <td>8</td>
      <td>4</td>
      <td>31</td>
    </tr>
    <tr>
      <th>3</th>
      <td>473</td>
      <td>3570</td>
      <td>473</td>
      <td>55</td>
      <td>55</td>
      <td>55</td>
      <td>55</td>
      <td>946</td>
      <td>200</td>
      <td>123</td>
      <td>2</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>25</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>30</td>
      <td>6</td>
      <td>8</td>
      <td>4</td>
      <td>31</td>
    </tr>
    <tr>
      <th>4</th>
      <td>343</td>
      <td>55</td>
      <td>5</td>
      <td>3399</td>
      <td>258</td>
      <td>212</td>
      <td>3498</td>
      <td>2095</td>
      <td>674</td>
      <td>2095</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>6</td>
      <td>9</td>
      <td>8</td>
      <td>4</td>
      <td>31</td>
    </tr>
  </tbody>
</table>

&#8195;Для выборки 150 пользователей расчитаем также некоторые другие показатели:
- Медианный день недели, в который началась сессия: 2
- Медианный час начала сессии: 13
- Медианное значение числа уникальных сайтов в сессиях: 7

&#8195;Дадим пользователям имена и ассоциируем с ними цвета и посмотрим на некоторые распределения.

<center>Распределение длины сессии в секундах</center>

&nbsp;

![ss](image4.png)

&nbsp;

<center>Распределения числа уникальных сайтов (#unique_sites) в сессии для каждого из 10 пользователей</center>

&nbsp;


![ss](image5.png)

&nbsp;

<center>Распределения часа начала сессии (start_hour) для каждого из 10 пользователей</center>

&nbsp;

![ss](image6.png)

&nbsp;

<center>Распределения дня недели, в который началась сессия (day_of_week) для каждого из 10 пользователей</center>

&nbsp;

![ss](image7.png)

<a id='4.2part'></a>
#### Вывод для каждого пользователя по построенным признакам

&nbsp;

&#8195;Сделаем вывод для каждого пользователя.

&nbsp;

&#8195;Ashley

&#8195;В сессии чаще всего 7 сайтов. Группировка вокруг 7ми с крупным выбросом на единице.
Намного больше сессий приходится на ранее утро.
Среда - любимый день. Суббота и воскресенье - нерабочие дни.
В сессии чаще всего 6 сайтов. Группировка вокруг 6ти.

&#8195;Avril

&#8195;Пики рабочего процесса в интернете приходятся на 15:00 и 17:00. Напомиает гауссиану с провалами.
Вторник - пик частоты активности. Среда, четверг, воскресенье - дни более частой активности.

&#8195;Bill

&#8195;Большинство сессий приходятся на 15:00, в два раза меньше, но тоже значительное количество на 8:30.
Понедельник, вторник - дни наиболее вероятной активности.

&#8195;Bob

&#8195;В сессии чаще всего 6 сайтов. Группировка вокруг 6ти со скосом в большую сторону.
Больше сессий с утра, да и вообще много сессий.
Вторник, чертверг, пятница- наиболее часто активные дни. Суббота и воскресенье - нерабочие дни.

&#8195;Dick

&#8195;Более-менее равномерное распределение сессий с тремя пиками
Понедельник, среда - редкая активность. Пятница - пик частоты активности. Далее частоты активности по убыванию: вторик, четверг, суббота. Воскресение - нерабочий день.

&#8195;Ed

&#8195;В сессии чаще всего 7 или 8 сайтов, так же очень часто 6. Далее по нисходящей.
Пик начала сессий на середине дня, сессий сравнительно немного. Напомиает гауссиану с крупными выбросами.
Среда, суббота, воскресение - пики частоты активности. Топ в среду. 

&#8195;John

&#8195;В сессии чаще всего 7 сайтов. Группировка вокруг 7ти со скосом в большую сторону.
Основные сессии приходятся на рабочие часы с пиками на 11 и 15 часов.
Частая активность во все рабочие дни со скосом в сторону пятницы. Суббота редкая активность. Воскресение - нерабочий день.


&#8195;Lindsey

&#8195;В сессии чаще всего 7 сайтов. Группировка вокруг 7ти со скосом в большую сторону.
Сессии сдвинуты в сторону первой половины дня.
Вторник - пик частоты активности. Далее по нисходящей до понедельника.

&#8195;Mary-Kate

&#8195;В сессии чаще всего 3 сайта. Минимальное количество с одним уникальным сайтом. В остальном распределением более-менее равномерное.
В целом больше сессий в сравнении с другими, при чем пик на вечернее время.
Суббота, воскресенье - концентраторы активности.

&#8195;Naomi

&#8195;В сессии чаще всего 6 сайтов. Группировка вокруг 6ти со скосом в большую сторону.
Пик сессий на 11 утра, 11-18 наиболее активное время.
Четверг - концентратор активности, далее вторник и среда.

<center>Топ 10 посещаемых сайтов</center>

&nbsp;

![ss](image8.png)

<a id='4.3part'></a>
#### Признаки "работает или развлекается пользователь"

&nbsp;

&#8195;Далее в рамках творческого подхода, в качестве критерия идентификации пользователя попробуем определять то, сколько что человек чаще делает: работает или развлекается. Конечно критерий сложный, но мы его только попытаемся симитировать.

&#8195;Для подсчета такого критерия мы можем считать факты частоту посещения определенных сайтов. Например почту можно считать условно за работу, а ютьюб за развлечения. С другой стороны ютьюб является наиболее частопосещаемым сайтом (очевидно, что люди в принципе предпочитают развлекаться), поэтому признак на его основе скорее может получиться неинформативным. Поэтому попробуем посчитать почту, ютьюб и фейсбук. Опять же фейсбук может быть связан с работой, но условно, с некоторыми допущениями, его все-таки можно взять в рубрику "я сейчас не работаю"

&#8195;В качестве признаков будем считать:

- Было ли посещение почтового ящика на mail.google.com
- Было ли посещение youtube
- Было ли посещение фейсбука
- Какова средняя длина сессиии на топ 30 сайтах

&#8195;Внешний вид массива с признаками:

&nbsp;

  <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>mail</th>
      <th>facebook</th>
      <th>top30_av_time</th>
      <th>youtube</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0</td>
      <td>0</td>
      <td>0.5</td>
      <td>0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0</td>
      <td>0</td>
      <td>0.0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0</td>
      <td>1</td>
      <td>7.0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0</td>
      <td>0</td>
      <td>0.0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>0</td>
      <td>1</td>
      <td>1.0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

&nbsp;

&#8195;Таблица со всеми признаками:

&nbsp;

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site1</th>
      <th>site2</th>
      <th>site3</th>
      <th>site4</th>
      <th>site5</th>
      <th>site6</th>
      <th>site7</th>
      <th>site8</th>
      <th>site9</th>
      <th>site10</th>
      <th>time_diff1</th>
      <th>time_diff2</th>
      <th>...</th>
      <th>time_diff7</th>
      <th>time_diff8</th>
      <th>time_diff9</th>
      <th>session_timespan</th>
      <th>#unique_sites</th>
      <th>start_hour</th>
      <th>day_of_week</th>
      <th>mail</th>
      <th>facebook</th>
      <th>top30_av_time</th>
      <th>youtube</th>
      <th>target</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>192</td>
      <td>576</td>
      <td>133</td>
      <td>3</td>
      <td>133</td>
      <td>133</td>
      <td>3</td>
      <td>133</td>
      <td>203</td>
      <td>133</td>
      <td>10</td>
      <td>0</td>
      <td>...</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>33</td>
      <td>5</td>
      <td>8</td>
      <td>4</td>
      <td>0</td>
      <td>0</td>
      <td>0.5</td>
      <td>0</td>
      <td>31</td>
    </tr>
    <tr>
      <th>1</th>
      <td>421</td>
      <td>193</td>
      <td>677</td>
      <td>254</td>
      <td>133</td>
      <td>31</td>
      <td>393</td>
      <td>3403</td>
      <td>217</td>
      <td>55</td>
      <td>1</td>
      <td>0</td>
      <td>...</td>
      <td>3</td>
      <td>3</td>
      <td>8</td>
      <td>284</td>
      <td>10</td>
      <td>8</td>
      <td>4</td>
      <td>0</td>
      <td>0</td>
      <td>0.0</td>
      <td>0</td>
      <td>31</td>
    </tr>
    <tr>
      <th>2</th>
      <td>55</td>
      <td>3</td>
      <td>55</td>
      <td>55</td>
      <td>5</td>
      <td>293</td>
      <td>421</td>
      <td>333</td>
      <td>899</td>
      <td>55</td>
      <td>0</td>
      <td>14</td>
      <td>...</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>258</td>
      <td>7</td>
      <td>8</td>
      <td>4</td>
      <td>0</td>
      <td>1</td>
      <td>7.0</td>
      <td>0</td>
      <td>31</td>
    </tr>
    <tr>
      <th>3</th>
      <td>473</td>
      <td>3570</td>
      <td>473</td>
      <td>55</td>
      <td>55</td>
      <td>55</td>
      <td>55</td>
      <td>946</td>
      <td>200</td>
      <td>123</td>
      <td>2</td>
      <td>1</td>
      <td>...</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>30</td>
      <td>6</td>
      <td>8</td>
      <td>4</td>
      <td>0</td>
      <td>0</td>
      <td>0.0</td>
      <td>0</td>
      <td>31</td>
    </tr>
    <tr>
      <th>4</th>
      <td>343</td>
      <td>55</td>
      <td>5</td>
      <td>3399</td>
      <td>258</td>
      <td>212</td>
      <td>3498</td>
      <td>2095</td>
      <td>674</td>
      <td>2095</td>
      <td>1</td>
      <td>0</td>
      <td>...</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>6</td>
      <td>9</td>
      <td>8</td>
      <td>4</td>
      <td>0</td>
      <td>1</td>
      <td>1.0</td>
      <td>0</td>
      <td>31</td>
    </tr>
  </tbody>
</table>

&#8195;Посмотрим на графики распределений построенных признаков.

&nbsp;

<center>Признак 'mail'</center>

![](image9.png)

&nbsp;

<center>Признак 'youtube'</center>

![](image10.png)

&nbsp;

<center>Признак 'facebook'</center>

![](image11.png)

&nbsp;

<center>Среднее время на топ-30 сайтов для каждого из 10 пользователей</center>

![](image12.png)


&#8195;Мы видим, что графики посещений сайтов значительно различаются, что может свидетельстовать о том, что нам удастся лучше идентифицировать пользователей с использованием такого набора дополнительных признаков. Среднее посещение сайта так же варьируется от пользователя к пользователю, а значит несет в себе информацию.

&#8195;В дальнейшем признаки будут протестированы на моделях.

<a id='5part'></a>
### 5. Сравнение алгоритмов и подбор модели

&#8195;Далее сравним на кросс-валидации несколько алгоритмов, разберемся, какие параметры длины сессии (session_length и window_size) работают эффективннее.

&#8195;Также для выбранного алгоритма проверим влияние дополнительных признаков на качество классификации, построим кривые валидации (как качество классификации зависит от одного из гиперпараметров алгоритма) и кривые обучения (как качество классификации зависит от объема выборки).

<a id='5.1part'></a>
#### Сравнение нескольких алгоритмов на сессиях из 10 сайтов

&nbsp;

&#8195;Разобьем выборку на 2 части. На одной будем проводить кросс-валидацию, на второй – оценивать модель, обученную после кросс-валидации.

&#8195;Тип кросс-валидации: 3-кратная, с перемешиванием, random_state=17.

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

&#8195;Обучим алгоритмы K-ближайших соседей, случайный лес, логистическую регрессию и метод опорных векторов с большинством гиперпараметров по-умолчанию. Параметры, а так же соответствующие значения метрик на кросс-валидации и на отложенной выборке приведены в таблице ниже.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg" style="undefined;table-layout: fixed; width: 700px">
<colgroup>
<col style="width: 200px">
<col style="width: 151px">
<col style="width: 180px">
</colgroup>
  <tr>
    <th class="tg-0lax">Алгоритм</th>
    <th class="tg-baqh"><span style="font-weight:700">Accuracy</span><br><br><br><span style="font-weight:700">на кросс-валидации</span></th>
    <th class="tg-baqh"><span style="font-weight:700">Accuracy</span><br><br><br><span style="font-weight:700">на отложенной выборке</span></th>
  </tr>
  <tr>
    <td class="tg-0pky">KNeighborsClassifier(n_neighbors=100)</td>
    <td class="tg-c3ow">0.565</td>
    <td class="tg-c3ow">0.584</td>
  </tr>
  <tr>
    <td class="tg-0pky">RandomForestClassifier(n_estimators=100,<br>random_state=17, oob_score=True)</td>
    <td class="tg-c3ow">0.721</td>
    <td class="tg-c3ow">0.73</td>
  </tr>
  <tr>
    <td class="tg-0pky">LogisticRegression(random_state=17)</td>
    <td class="tg-c3ow">0.761</td>
    <td class="tg-c3ow">0.782</td>
  </tr>
  <tr>
    <td class="tg-0lax">LinearSVC(C=1, random_state=17)</td>
    <td class="tg-baqh">0.753</td>
    <td class="tg-baqh">0.777</td>
  </tr>
</table>

&#8195;Далее подберем гиперпараметр C, который отвечает за регуляризацию у логистической регрессии и SVM.

In [252]:
with open('image13.png', 'rb') as f:
    img13 = f.read()
with open('image14.png', 'rb') as f:
    img14 = f.read()
with open('image15.png', 'rb') as f:
    img15 = f.read()
with open('image16.png', 'rb') as f:
    img16 = f.read()
accordion = widgets.Accordion(children=[widgets.Image(value=img13), widgets.Image(value=img14),
                              widgets.Image(value=img15), widgets.Image(value=img16)])
accordion.set_title(0, 'Кривая валидации для значений C логистической регрессии в диапазоне 1e-4 до 1e2:')
accordion.set_title(1, 'Кривая валидации для значений C логистической регрессии в диапазоне 0.1 до 7:')
accordion.set_title(2, 'Кривая валидации для значений C алгоритма SVM в диапазоне 1e-4 до 1e4:')
accordion.set_title(3, 'Кривая валидации для значений C алгоритма SVM в диапазоне 1e-3 до 1')
accordion

Accordion(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01w\x00\x00\x00\xfc\x08\x06\x00…

In [248]:
w1 = widgets.SelectionSlider(
    options=['1e-4 : 1e2', '0.1 : 7'],
    value='1e-4 : 1e2',
    description='C in range',
    disabled=False,
    orientation='horizontal',
    readout=True
)
def f1(x):
    if x == '1e-4 : 1e2':
        accuracy, C = 0.7610, 2.782
    if x == '0.1 : 7':
        accuracy, C, accuracyvalid = 0.7613, 1.553, 0.784
        print('Лучшее значение доли на отложенной выборке: {}'.format(accuracyvalid))
    print('Лучшее значение доли на CV: {}'.format(accuracy))
    print('C при максимальном значении доли: {}'.format(C))
    
w2 = widgets.SelectionSlider(
    options=['1e-4 : 1e2', '1e-3 : 1'],
    value='1e-4 : 1e2',
    description='C in range',
    disabled=False,
    orientation='horizontal',
    readout=True
)
def f2(x):
    if x == '1e-4 : 1e2':
        accuracy, C = 0.685, 3333.333
    if x == '1e-3 : 1':
        accuracy, C, accuracyvalid = 0.765, 0.104, 0.781
        print('Лучшее значение доли на отложенной выборке: {}'.format(accuracyvalid))
        
    print('Лучшее значение доли на CV: {}'.format(accuracy))
    print('C при максимальном значении доли: {}'.format(C))

In [249]:
print('Подбор параметров для logregression:')
interact(f1, x=w1);
print('Подбор параметров для SVM:')
interact(f2, x=w2);

Подбор параметров для logregression:


interactive(children=(SelectionSlider(description='C in range', options=('1e-4 : 1e2', '0.1 : 7'), value='1e-4…

Подбор параметров для SVM:


interactive(children=(SelectionSlider(description='C in range', options=('1e-4 : 1e2', '1e-3 : 1'), value='1e-…

&#8195;SMV показывает лучший результат на кросс-валидации, при проигрыше на отложенной выборке. Так или иначе далее поработаем с SVM в процессе подбора оптимальной длины сессии и ширины окна.

<a id='5.2part'></a>
#### Выбор параметров – длины сессии и ширины окна

&#8195;Определим функцию model_assessment, с помощью которой измерим метрику accuracy при различных длинах сессии и ширины окна.

&#8195;Применять функцию будем к модели SVM с параметром C = 0.104 (давшем лучшее качество на кросс-валидации).

&#8195;Ниже представлен отчет по проведенному перебору.

&#8195;Результаты для выборок 10 и 150 пользователей:

In [250]:
# 10 users:

with open('report_10users_logreg_c_select.txt') as f:
    lst = f.readlines()
with open('report_10users_logreg_c_select_best.txt') as f:
    best = f.readlines()
w3 = widgets.ToggleButtons(
    options=['Лучший вариант', 'Список переборов'],
    description='10 us. sample:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

def f3(x):
    if x == 'Список переборов':
        for i in lst:
            print(i)
    if x == 'Лучший вариант':
        for i in best:
            print(i)
            
interact(f3, x=w3);

# 150 users:

with open('report_150users_logreg_c_select.txt') as f:
    lst2 = f.readlines()
with open('report_150users_logreg_c_select_best.txt') as f:
    best2 = f.readlines()
w4 = widgets.ToggleButtons(
    options=['Лучший вариант', 'Список переборов'],
    description='150 us. sample:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

def f4(x):
    if x == 'Список переборов':
        for i in lst2:
            print(i)
    if x == 'Лучший вариант':
        for i in best2:
            print(i)
            
interact(f4, x=w4);

interactive(children=(ToggleButtons(description='10 us. sample:', options=('Лучший вариант', 'Список переборов…

interactive(children=(ToggleButtons(description='150 us. sample:', options=('Лучший вариант', 'Список переборо…

&#8195;На выборке 10 пользователей заметно, что увеличение длины сессии и уменьшение ширины окна увеличивает качество на кросс-валидации и отложенной выборке. Что логично, учитывая что при таких параметрах получается выборка максимального размера.

&#8195;Кроме того качество многоклассовой классификации на выборке 150 пользователей оставляет желать много лучшего. Поэтому следующим этапом станет идентификация конкретного пользователя или бинарная классификация.

<a id='5.3part'></a>
#### Идентификация конкретного пользователя и кривые обучения

&#8195;Обучим LogisticRegressionCV с C = 1.553 (лучшим на кросс-валидации для 10 пользователей). 

&#8195;Посмотрим на средние доли правильных ответов на кросс-валидации в задаче идентификации каждого пользователя по отдельности.

In [251]:
with open('report_150users_logreg_ovr_predict.txt') as f:
     lst = f.read()
accordion = widgets.Accordion(children=[widgets.Textarea(lst)])
accordion.set_title(0, 'Доли правильных ответов на кросс-валидации по пользователям')
accordion

Accordion(children=(Textarea(value='User 0006, CV score: 0.9960795612888451\nUser 0013, CV score: 0.9963926034…

&#8195;Стоит помнить про имеющийся дисбаланс классов, так что высокую долю правильных ответов можно получить константным прогнозом. 

&#8195;Проверим лучше ли имеющийся алгоритм константного. Для этого посчитаем для каждого пользователя разницу между долей правильных ответов на кросс-валидации и долей меток в y_train_150, отличных от ID этого пользователя (именно такую долю правильных ответов можно получить, если классификатор всегда "говорит", что это не пользователь с номером 𝑖 в задаче классификации 𝑖-vs-All).



&#8195;Доля пользователей, для которых логистическая регрессия на кросс-валидации дает прогноз лучше константного: 

&nbsp;

<center>0.82

&#8195;Построим кривую обучения, что бы понять, можно ли улучшить качество классификации, увеличив выборку.

&#8195;Таким образом посчитаем доли правильных ответов на кросс-валидации в задаче классификации "user128-vs-All" в зависимости от размера выборки с помощью learning_curve и посмотрим на график plot_learning_curve.

&nbsp;

![ss](image17.png)

&nbsp;

&#8195;По графику наблюдается прекращение роста кривой при 50000 значениях в выборке, улучшение результата при дальнейшем увеличении выборки является маловероятным.

<a id='6part'></a>
### 6. Соревнование Kaggle

<a id='6.1part'></a>
#### Анализ и подготовка данных

&nbsp;

В обучающей выборке видим следующие признаки:

- site1 – индекс первого посещенного сайта в сессии
- time1 – время посещения первого сайта в сессии
- ...
- site10 – индекс 10-го посещенного сайта в сессии
- time10 – время посещения 10-го сайта в сессии
- user_id – ID пользователя
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут.

В тестовой выборке все тоже самое, просто без столбца user_id.

&nbsp;

Посмотрим на статистику признаков.

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил vk.com в 20:01, потом yandex.ru в 20:29, затем google.com в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта vk.com, time1 – 2015-01-01 20:01:00, site2 – ID сайта yandex.ru, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с google.com пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения vk.com.

&nbsp;

В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.

Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.

Получим таблицу:

&nbsp;

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site1</th>
      <th>site2</th>
      <th>site3</th>
      <th>site4</th>
      <th>site5</th>
      <th>site6</th>
      <th>site7</th>
      <th>site8</th>
      <th>site9</th>
      <th>site10</th>
    </tr>
    <tr>
      <th>session_id</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>718</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>890</td>
      <td>941</td>
      <td>3847</td>
      <td>941</td>
      <td>942</td>
      <td>3846</td>
      <td>3847</td>
      <td>3846</td>
      <td>1516</td>
      <td>1518</td>
    </tr>
    <tr>
      <th>3</th>
      <td>14769</td>
      <td>39</td>
      <td>14768</td>
      <td>14769</td>
      <td>37</td>
      <td>39</td>
      <td>14768</td>
      <td>14768</td>
      <td>14768</td>
      <td>14768</td>
    </tr>
    <tr>
      <th>4</th>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
    </tr>
    <tr>
      <th>5</th>
      <td>22</td>
      <td>177</td>
      <td>175</td>
      <td>178</td>
      <td>177</td>
      <td>178</td>
      <td>175</td>
      <td>177</td>
      <td>177</td>
      <td>178</td>
    </tr>
    <tr>
      <th>6</th>
      <td>570</td>
      <td>21</td>
      <td>570</td>
      <td>21</td>
      <td>21</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>7</th>
      <td>803</td>
      <td>23</td>
      <td>5956</td>
      <td>17513</td>
      <td>37</td>
      <td>21</td>
      <td>803</td>
      <td>17514</td>
      <td>17514</td>
      <td>17514</td>
    </tr>
    <tr>
      <th>8</th>
      <td>22</td>
      <td>21</td>
      <td>29</td>
      <td>5041</td>
      <td>14422</td>
      <td>23</td>
      <td>21</td>
      <td>5041</td>
      <td>14421</td>
      <td>14421</td>
    </tr>
    <tr>
      <th>9</th>
      <td>668</td>
      <td>940</td>
      <td>942</td>
      <td>941</td>
      <td>941</td>
      <td>942</td>
      <td>940</td>
      <td>23</td>
      <td>21</td>
      <td>22</td>
    </tr>
    <tr>
      <th>10</th>
      <td>3700</td>
      <td>229</td>
      <td>570</td>
      <td>21</td>
      <td>229</td>
      <td>21</td>
      <td>21</td>
      <td>21</td>
      <td>2336</td>
      <td>2044</td>
    </tr>
  </tbody>
</table>

<a id='6.2part'></a>
#### Первый сабмит

&nbsp;

Создадим из тестовой и обучающей выборок разреженные матрицы X_train_sparse и X_test_sparse аналогичным образом как ранее, что бы использовать индексы сайтов как признаки. Столбец обучающей выборки с целевыми метками выведем в отдельный вектор.

Обратим внимание и на то, что в сессиях меньше 10 сайтов у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных. Поэтому первый столбец разреженной матрицы удалим.

&nbsp;


<center>Размерности полученных матриц
    
&nbsp;


<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-cly1{text-align:left;vertical-align:middle}
</style>
<table class="tg">
  <tr>
    <th class="tg-cly1"></th>
    <th class="tg-cly1">Строк</th>
    <th class="tg-cly1">Столбцов</th>
  </tr>
  <tr>
    <td class="tg-cly1">Обучающая выборка</td>
    <td class="tg-cly1">253561 </td>
    <td class="tg-cly1">48370</td>
  </tr>
  <tr>
    <td class="tg-cly1">Тестовая выборка</td>
    <td class="tg-cly1">82797 </td>
    <td class="tg-cly1">48370</td>
  </tr>
</table>

Разобьем обучающую выборку на 2 части в пропорции 7/3, причем не перемешивая. Исходные данные упорядочены по времени, тестовая выборка по времени четко отделена от обучающей, сохраним теже условия.

    train_share = int(.7 * X_train_sparse.shape[0])
    X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
    X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

Получим соответствующие пары  X_train, y_train и X_valid, y_valid.

Создадим объект sklearn.linear_model.SGDClassifier с логистической функцией потерь. Остальные параметры оставим по умолчанию. 
Обучим модель на выборке (X_train, y_train), сделаем предсказание на (X_valid, y_valid).
    
    sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
    sgd_logit.fit(X_train, y_train)

&nbsp;

<center>roc_auc_score = 0.934

Сформируем прогноз для отправки на kaggle: cделаем прогноз в виде предсказанных вероятностей (predict_proba) отнесения к классу 1 для тестовой выборки с помощью той же модели, но обученной уже на всей обучающей выборке.
Отправим прогноз на kaggle от имени команды '[YDF & MIPT] Coursera_Dmi2ry'

&nbsp;

<center>Оценка kaggle = 0.91275

<a id='6.3part'></a>
#### Второй сабмит

&nbsp;

Обучим логрегрессию на выборке с дополнительными признаками.

Для начала добавим четыре признака, которые мы уже однажды тестировали в части 4:
- 'session_timespan' - длина сессии
- 'day_of_week' - день старта сессии
- 'start_hour' - час старта сессии
- 'unique_sites' - количество уникальных сайтов в сессии

Создадим функцию, которая посчитает вышеперечисленные признаки и сформирует матрицу. На вход будем подавать train_test_df из конкатенированных ранее матриц train_sessions.csv и test_sessions.csv, на выходе будем получать матрицу следующего вида:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site1</th>
      <th>site10</th>
      <th>site2</th>
      <th>site3</th>
      <th>site4</th>
      <th>site5</th>
      <th>site6</th>
      <th>site7</th>
      <th>site8</th>
      <th>site9</th>
      <th>...</th>
      <th>time4</th>
      <th>time5</th>
      <th>time6</th>
      <th>time7</th>
      <th>time8</th>
      <th>time9</th>
      <th>session_timespan</th>
      <th>day_of_week</th>
      <th>start_hour</th>
      <th>unique_sites</th>
    </tr>
    <tr>
      <th>session_id</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>718</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>...</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>3</td>
      <td>10</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2</th>
      <td>890</td>
      <td>1518</td>
      <td>941</td>
      <td>3847</td>
      <td>941</td>
      <td>942</td>
      <td>3846</td>
      <td>3847</td>
      <td>3846</td>
      <td>1516</td>
      <td>...</td>
      <td>2014-02-22 11:19:51</td>
      <td>2014-02-22 11:19:51</td>
      <td>2014-02-22 11:19:51</td>
      <td>2014-02-22 11:19:52</td>
      <td>2014-02-22 11:19:52</td>
      <td>2014-02-22 11:20:15</td>
      <td>26</td>
      <td>5</td>
      <td>11</td>
      <td>7</td>
    </tr>
    <tr>
      <th>3</th>
      <td>14769</td>
      <td>14768</td>
      <td>39</td>
      <td>14768</td>
      <td>14769</td>
      <td>37</td>
      <td>39</td>
      <td>14768</td>
      <td>14768</td>
      <td>14768</td>
      <td>...</td>
      <td>2013-12-16 16:40:19</td>
      <td>2013-12-16 16:40:19</td>
      <td>2013-12-16 16:40:19</td>
      <td>2013-12-16 16:40:20</td>
      <td>2013-12-16 16:40:21</td>
      <td>2013-12-16 16:40:22</td>
      <td>7</td>
      <td>0</td>
      <td>16</td>
      <td>4</td>
    </tr>
    <tr>
      <th>4</th>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>...</td>
      <td>2014-03-28 10:53:42</td>
      <td>2014-03-28 10:54:12</td>
      <td>2014-03-28 10:54:42</td>
      <td>2014-03-28 10:55:12</td>
      <td>2014-03-28 10:55:42</td>
      <td>2014-03-28 10:56:12</td>
      <td>270</td>
      <td>4</td>
      <td>10</td>
      <td>1</td>
    </tr>
    <tr>
      <th>5</th>
      <td>22</td>
      <td>178</td>
      <td>177</td>
      <td>175</td>
      <td>178</td>
      <td>177</td>
      <td>178</td>
      <td>175</td>
      <td>177</td>
      <td>177</td>
      <td>...</td>
      <td>2014-02-28 10:55:23</td>
      <td>2014-02-28 10:55:23</td>
      <td>2014-02-28 10:55:59</td>
      <td>2014-02-28 10:55:59</td>
      <td>2014-02-28 10:55:59</td>
      <td>2014-02-28 10:57:06</td>
      <td>246</td>
      <td>4</td>
      <td>10</td>
      <td>4</td>
    </tr>
  </tbody>
</table>

Преобразуем получившиеся признаки так, что на них можно обучаться - ведь в матрице выше мы имеем категориальные, на которых методы машинного обучения не работают. Для признаков-сайтов используем уже неоднократно используювшуюся функцию sparsecreator, а для преобразования вновь созданных признаков будет удобно применить pd.get_dummies

    X_sparse = sparsecreator(X.iloc[:, :10].values)
    X_sparse = X_sparse[:, 1:]
    dummies = pd.get_dummies(X.iloc[:, -4:], sparse=True, columns=['start_hour', 'unique_sites', 'day_of_week','session_timespan'])
    X_sparse_dum = hstack([X_sparse, csr_matrix(dummies)], format='csr')
    
 Разделим X_sparse_dum на обучение и тест:
 
    X_train_sparse = X_sparse_dum[:253561]
    X_test_sparse = X_sparse_dum[253561:]
    
Теперь разобьем X_test_sparse на обучение и тест, обучим классификатор и посчитаем площадь под ROC-кривой:

&nbsp;


<center>roc_auc_score = 0.971

&nbsp;

Результат выглядит неплохо. Попробуем обучить классификатор на полной выборке X_train_sparse и получить вектор предсказаний меток для X_test_sparse. Сформируем прогноз в виде вероятностей (predict_proba) для отправки на kaggle. Сделаем сабмит.

&nbsp;

<center>Оценка kaggle = 0.93562

<a id='6.4part'></a>
#### Третий сабмит
Попробуем добавить признаки, которые направлены на анализ деятельности пользователя: работает он или отдыхает, соответственно которые могут показать как часто он делает первое или второе.

Добавим признаки-флажки посещения того или иного сайта в течении сессии:

- 'mail'
- 'facebook'
- 'youtube'

Получим матрицу следующей структуры:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site1</th>
      <th>site10</th>
      <th>site2</th>
      <th>site3</th>
      <th>site4</th>
      <th>site5</th>
      <th>site6</th>
      <th>site7</th>
      <th>site8</th>
      <th>site9</th>
      <th>...</th>
      <th>time7</th>
      <th>time8</th>
      <th>time9</th>
      <th>session_timespan</th>
      <th>day_of_week</th>
      <th>start_hour</th>
      <th>unique_sites</th>
      <th>mail</th>
      <th>facebook</th>
      <th>youtube</th>
    </tr>
    <tr>
      <th>session_id</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>718</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>...</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>3</td>
      <td>10</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>890</td>
      <td>1518</td>
      <td>941</td>
      <td>3847</td>
      <td>941</td>
      <td>942</td>
      <td>3846</td>
      <td>3847</td>
      <td>3846</td>
      <td>1516</td>
      <td>...</td>
      <td>2014-02-22 11:19:52</td>
      <td>2014-02-22 11:19:52</td>
      <td>2014-02-22 11:20:15</td>
      <td>26</td>
      <td>5</td>
      <td>11</td>
      <td>7</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>14769</td>
      <td>14768</td>
      <td>39</td>
      <td>14768</td>
      <td>14769</td>
      <td>37</td>
      <td>39</td>
      <td>14768</td>
      <td>14768</td>
      <td>14768</td>
      <td>...</td>
      <td>2013-12-16 16:40:20</td>
      <td>2013-12-16 16:40:21</td>
      <td>2013-12-16 16:40:22</td>
      <td>7</td>
      <td>0</td>
      <td>16</td>
      <td>4</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>782</td>
      <td>...</td>
      <td>2014-03-28 10:55:12</td>
      <td>2014-03-28 10:55:42</td>
      <td>2014-03-28 10:56:12</td>
      <td>270</td>
      <td>4</td>
      <td>10</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>5</th>
      <td>22</td>
      <td>178</td>
      <td>177</td>
      <td>175</td>
      <td>178</td>
      <td>177</td>
      <td>178</td>
      <td>175</td>
      <td>177</td>
      <td>177</td>
      <td>...</td>
      <td>2014-02-28 10:55:59</td>
      <td>2014-02-28 10:55:59</td>
      <td>2014-02-28 10:57:06</td>
      <td>246</td>
      <td>4</td>
      <td>10</td>
      <td>4</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

Повторим все теже шаги, что и с предыдущим сабмитом и замерим roc-auc на отложенной выборке.

&nbsp;

<center>ROC_AUC = 0.973

&nbsp;

Результат получился немного выше. Попробуем обучить классификатор на полной выборке X_train_sparse и получить вектор предсказаний меток для X_test_sparse. Сформируем прогноз в виде вероятностей (predict_proba) для отправки на kaggle. Сделаем сабмит.

&nbsp;

<center>Оценка kaggle = 0.93734

&nbsp;

Результат получился выше чем 0.93562, так что можно говорить об успешности применения дополнительных признаков, так как не смотря на относительно небольшой прирост, вычислительные затраты на расчет этих признаков так же небольшие.